In [240]:
import pandas as pd
import json
import re
from urllib.parse import urlparse

with open('/Users/timkh/webassembl/outguard/instrumentation/js_fingerprint/output.json', 'r') as wap_json:
    wappalyzer = json.load(wap_json)
result = pd.read_csv('/Users/timkh/webassembl/outguard/parser/wasmsite_parsed.csv')

In [274]:
pattern = r'jsecoin|coinhive|deepminer|coinimp|crypto-loot|projectpoi|coinlab|cloudcoins|monerominer|coinhave|monero\.cc|webmine|inwemo'

In [243]:
with open('/Users/timkh/webassembl/outguard/resources/miners-fingerprinting.txt', 'r') as fi:
    finger = fi.readlines()

In [262]:
finger = pd.Series(finger).str.split('http://www.').str[1].str.split('/\t').str[0]

In [59]:
with open('/Users/timkh/webassembl/outguard/instrumentation/alive_urls.txt', 'r') as f:
    url = f.readlines()

In [149]:
alive_df = pd.DataFrame(columns= result.columns, index=[0])

In [150]:
for i in range(len(result)):
    for j in range(len(url)):
        if result.url_path[i].split('-1702')[0] == url[j].split('\n')[0]:
            alive_df.loc[i] = result.iloc[i]

In [192]:
alive_df['url'] = alive_df.url_path.str.split('-170').str[0]
alive_df = alive_df.drop_duplicates(['url'], keep='first')

In [303]:
alive_df.label.value_counts()

0    2587
Name: label, dtype: int64

In [332]:
alive_df['label'] = 0
for key, val in pd.DataFrame(wappalyzer).urls.items():
    ul = urlparse(str(list(val.keys())[0])).hostname
    for i in range(len(alive_df)):
        # print(list(val.keys())[0])
        if ((ul == alive_df['url'].iloc[i]) & (re.search(pattern, ul, re.IGNORECASE) is not None)):
            alive_df['label'].iloc[i] = 1
        elif ((ul == alive_df['url'].iloc[i]) & ((ul in list(finger)) == True)):
            alive_df['label'].iloc[i] = 1

/var/folders/cm/t1j_h30d34sbggmtvprdjqk80000gn/T/ipykernel_24699/2533214085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alive_df['label'] = 0
/var/folders/cm/t1j_h30d34sbggmtvprdjqk80000gn/T/ipykernel_24699/2533214085.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alive_df['label'].iloc[i] = 1
/var/folders/cm/t1j_h30d34sbggmtvprdjqk80000gn/T/ipykernel_24699/2533214085.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [337]:
alive_df.label.value_counts()

0    2559
1      28
Name: label, dtype: int64